In [ ]:
%matplotlib inline
from pyvista import set_plot_theme

set_plot_theme("document")

# PyVista & VTK

Show how PyVista uses VTK and how you can combine the best of both worlds!

## Transitioning from VTK to PyVista

VTK is primarily developed in C++ and uses chained setter and getter commands to access data. Instead, PyVista wraps the VTK data types into numpy arrays so that users can benefit from its bracket syntax and fancy indexing. This section demonstrates the difference between the two approaches in a series of examples.

For example, to hard-code points for a vtk.vtkImageData data structure using VTK Python’s bindings, one would write the following:

In [ ]:
from math import cos, sin

import vtk

# Create (x, y) points for a 300x300 image dataset

points = vtk.vtkDoubleArray()
points.SetName("points")
points.SetNumberOfComponents(1)
points.SetNumberOfTuples(300 * 300)

for x in range(300):
    for y in range(300):
        points.SetValue(x * 300 + y, 127.5 + (1.0 + sin(x / 25.0) * cos(y / 25.0)))

# Create the image structure

image_data = vtk.vtkImageData()
image_data.SetOrigin(0, 0, 0)
image_data.SetSpacing(1, 1, 1)
image_data.SetDimensions(300, 300, 1)

# Assign the points to the image

image_data.GetPointData().SetScalars(points)

As you can see, there is quite a bit of boilerplate that goes into the creation of a simple [vtk.vtkImageData](https://vtk.org/doc/nightly/html/classvtkImageData.html) dataset. PyVista provides much more concise syntax that is more “Pythonic”. The equivalent code in PyVista is:

In [ ]:
import numpy as np
import pyvista

# Use the meshgrid function to create 2D "grids" of the x and y values.
# This section effectively replaces the vtkDoubleArray.

xi = np.arange(300)
x, y = np.meshgrid(xi, xi)
values = 127.5 + (1.0 + np.sin(x / 25.0) * np.cos(y / 25.0))

# Create the grid.  Note how the values must use Fortran ordering.

grid = pyvista.UniformGrid(dims=(300, 300, 1))
grid.point_data["values"] = values.flatten(order="F")

Here, PyVista has done several things for us:
1. PyVista combines the dimensionality of the data (in the shape of the [numpy.ndarray](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html#numpy.ndarray)) with the values of the data in one line. VTK uses “tuples” to describe the shape of the data (where it sits in space) and “components” to describe the type of data (1 = scalars/scalar fields, 2 = vectors/vector fields, n = tensors/tensor fields). Here, shape and values are stored concretely in one variable.
1. [pyvista.UniformGrid](https://docs.pyvista.org/api/core/_autosummary/pyvista.UniformGrid.html#pyvista.UniformGrid) wraps [vtk.vtkImageData](https://vtk.org/doc/nightly/html/classvtkImageData.html), just with a different name; they are both containers of evenly spaced points. Your data does not have to be an “image” to use it with vtk.vtkImageData; rather, like images, values in the dataset are evenly spaced apart like pixels in an image.

   Furthermore, since we know the container is for uniformly spaced data, pyvista sets the origin and spacing by default to `(0, 0, 0)` and `(1, 1, 1)`. This is another great thing about PyVista and Python! Rather than having to know everything about the VTK library up front, you can get started very easily! Once you get more familiar with it and need to do something more complex, you can dive deeper. For example, changing the origin and spacing is as simple as:

In [ ]:
grid.origin = (10, 20, 10)
grid.spacing = (2, 3, 5)

3. The name for the point_array is given directly in dictionary-style fashion. Also, since VTK stores data on the heap (linear segments of RAM; a C++ concept), the data must be flattened and put in Fortran ordering (which controls how multidimensional data is laid out in physically 1d memory; numpy uses “C”-style memory layout by default). This is why in our earlier example, the first argument to SetValue() was written as x*300 + y. Here, numpy takes care of this for us quite nicely and it’s made more explicit in the code, following the Python best practice of “Explicit is better than implicit”.

Finally, with PyVista, each geometry class contains methods that allow you to immediately plot the mesh without also setting up the plot. For example, in VTK you would have to do:

In [ ]:
actor = vtk.vtkImageActor()
actor.GetMapper().SetInputData(image_data)
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
renWin.SetWindowName("ReadSTL")
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)
ren.AddActor(actor)
iren.Initialize()
renWin.Render()
iren.Start()

However, with PyVista you only need:

In [ ]:
grid.plot(cpos="xy", show_scalar_bar=False, cmap="coolwarm")

## PointSet Construction

In [ ]:
import vtk

vtk_array = vtk.vtkDoubleArray()
vtk_array.SetNumberOfComponents(3)
vtk_array.SetNumberOfValues(9)
vtk_array.SetValue(0, 0)
vtk_array.SetValue(1, 0)
vtk_array.SetValue(2, 0)
vtk_array.SetValue(3, 1)
vtk_array.SetValue(4, 0)
vtk_array.SetValue(5, 0)
vtk_array.SetValue(6, 0.5)
vtk_array.SetValue(7, 0.667)
vtk_array.SetValue(8, 0)
vtk_points = vtk.vtkPoints()
vtk_points.SetData(vtk_array)
print(vtk_points)

To do the same within PyVista, you simply need to create a NumPy array:

In [ ]:
import numpy as np

np_points = np.array([[0, 0, 0], [1, 0, 0], [0.5, 0.667, 0]])

You can use [pyvista.vtk_points()](https://docs.pyvista.org/api/utilities/_autosummary/pyvista.vtk_points.html#pyvista.vtk_points) to construct a [vtkPoints](https://vtk.org/doc/nightly/html/classvtkPoints.html) object, but this is unnecessary in almost all situations.

Since the end goal is to construct a [pyvista.DataSet](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.html#pyvista.DataSet), you would simply pass the `np_points` array to the [pyvista.PolyData](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.html#pyvista.PolyData) constructor:

In [ ]:
import pyvista

poly_data = pyvista.PolyData(np_points)

Whereas in VTK you would have to do:

In [ ]:
vtk_poly_data = vtk.vtkPolyData()
vtk_poly_data.SetPoints(vtk_points)

The same goes with assigning face or cell connectivity/topology. With VTK you would normally have to loop using `InsertNextCell()` and `InsertCellPoint()`. For example, to create a single cell (triangle) and then assign it to [vtkPolyData](https://vtk.org/doc/nightly/html/classvtkPolyData.html):

In [ ]:
cell_arr = vtk.vtkCellArray()
cell_arr.InsertNextCell(3)
cell_arr.InsertCellPoint(0)
cell_arr.InsertCellPoint(1)
cell_arr.InsertCellPoint(2)
vtk_poly_data.SetPolys(cell_arr)

In PyVista, we can assign this directly in the constructor and then access it (or change it) from the [faces](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.faces.html#pyvista.PolyData.faces) attribute.

In [ ]:
faces = np.array([3, 0, 1, 2])
poly_data = pyvista.PolyData(np_points, faces)
poly_data.faces

## Object Representation

Both VTK and PyVista provide representations for their objects.

VTK provides a verbose representation (useful for debugging) of their datatypes that can be accessed via [print()](https://docs.python.org/dev/library/functions.html#print), as the ``__repr__`` (unlike ``__str__``) only provides minimal information about each object:

In [ ]:
print(vtk_poly_data)

PyVista chooses to show minimal data in the `repr()`, preferring explicit attribute access on meshes for the bulk of attributes. For example:

In [ ]:
poly_data

In this representation we see:

- Number of cells [n_cells](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.n_cells.html#pyvista.DataSet.n_cells)
- Number of points [n_points](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.n_points.html#pyvista.DataSet.n_points)
- Bounds of the mesh [bounds](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.bounds.html#pyvista.DataSet.bounds)
- Number of data arrays [n_arrays](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.n_arrays.html#pyvista.DataSet.n_arrays)

All other attributes like [lines](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.lines.html#pyvista.PolyData.lines), [point_data](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.point_data.html#pyvista.DataSet.point_data), or [cell_data](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.cell_data.html#pyvista.DataSet.cell_data) can be accessed directly from the object. This approach was chosen to allow for a brief summary showing key parts of the [DataSet](https://docs.pyvista.org/api/core/_autosummary/pyvista.DataSet.html#pyvista.DataSet) without overwhelming the user.

## Tradeoffs

While most features can, not everything can be simplified without losing functionality or performance.

In the [collision](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyDataFilters.collision.html#pyvista.PolyDataFilters.collision) filter, we demonstrate how to calculate the collision between two meshes. For example:

In [ ]:
import pyvista

# create a default sphere and a shifted sphere
mesh_a = pyvista.Sphere()
mesh_b = pyvista.Sphere(center=(-0.4, 0, 0))
out, n_coll = mesh_a.collision(mesh_b, generate_scalars=True, contact_mode=2)

pl = pyvista.Plotter()
pl.add_mesh(out)
pl.add_mesh(mesh_b, style="wireframe", color="k")
pl.camera_position = "xy"
pl.show()

Under the hood, the collision filter detects mesh collisions using oriented bounding box (OBB) trees. For a single collision, this filter is as performant as the VTK counterpart, but when computing multiple collisions with the same meshes, as in the [Collision](https://docs.pyvista.org/examples/01-filter/collisions.html#collision-example) example, it is more efficient to use the [vtkCollisionDetectionFilter](https://vtk.org/doc/nightly/html/classvtkCollisionDetectionFilter.html), as the OBB tree is computed once for each mesh. In most cases, pure PyVista is sufficient for most data science, but there are times when you may want to use VTK classes directly.

Note that nothing stops you from using VTK classes and then wrapping the output with PyVista. For example:

In [ ]:
import pyvista
import vtk

# Create a circle using vtk
polygonSource = vtk.vtkRegularPolygonSource()
polygonSource.GeneratePolygonOff()
polygonSource.SetNumberOfSides(50)
polygonSource.SetRadius(5.0)
polygonSource.SetCenter(0.0, 0.0, 0.0)
polygonSource.Update()

# wrap and plot using pyvista
mesh = pyvista.wrap(polygonSource.GetOutput())
mesh.plot(line_width=3, cpos="xy", color="k")

In this manner, you can get the “best of both worlds” should you need the flexibility of PyVista and the raw power of VTK.

You can use [pyvista.PolyData()](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.html#pyvista-polydata) for a one line replacement of the above VTK code.

### Exercise

Let's use [pyvista.PolyData()](https://docs.pyvista.org/api/core/_autosummary/pyvista.PolyData.html#pyvista-polydata) to replace the above code with Pythonic.

In [ ]:
# Your code here

In [ ]:
import pyvista

# We still not wrap the `GeneratePolygonOff` method. See https://github.com/pyvista/pyvista/pull/2767 .
mesh = pyvista.Polygon(n_sides=50, radius=5.0, center=[0.0, 0.0, 0.0])
mesh.plot(line_width=3, cpos="xy", color="k")

## Using Common Filters

Using common filters like thresholding and clipping.

In [ ]:
import pyvista as pv
from pyvista import examples

PyVista wrapped data objects have a suite of common filters ready for
immediate use directly on the object. These filters include the
following (see [filters_ref](https://docs.pyvista.org/api/core/filters.html#filters-ref) for a
complete list):

-   `slice`: creates a single slice through the input dataset on a user
    defined plane
-   `slice_orthogonal`: creates a `MultiBlock` dataset of three
    orthogonal slices
-   `slice_along_axis`: creates a `MultiBlock` dataset of many slices
    along a specified axis
-   `threshold`: Thresholds a dataset by a single value or range of
    values
-   `threshold_percent`: Threshold by percentages of the scalar range
-   `clip`: Clips the dataset by a user defined plane
-   `outline_corners`: Outlines the corners of the data extent
-   `extract_geometry`: Extract surface geometry

To use these filters, call the method of your choice directly on your
data object:


In [ ]:
dataset = examples.load_uniform()
dataset.set_active_scalars("Spatial Point Data")

# Apply a threshold over a data range
threshed = dataset.threshold([100, 500])

outline = dataset.outline()

And now there is a thresholded version of the input dataset in the new
`threshed` object. To learn more about what keyword arguments are
available to alter how filters are executed, print the docstring for any
filter attached to PyVista objects with either `help(dataset.threshold)`
or using `shift+tab` in an IPython environment.

In [ ]:
help(dataset.threshold)

We can now plot this filtered dataset along side an outline of the
original dataset

In [ ]:
p = pv.Plotter()
p.add_mesh(outline, color="k")
p.add_mesh(threshed)
p.camera_position = [-2, 5, 3]
p.show()

What about other filters? Let\'s collect a few filter results and
compare them:


In [ ]:
contours = dataset.contour()
slices = dataset.slice_orthogonal()
glyphs = dataset.glyph(factor=1e-3, geom=pv.Sphere())

p = pv.Plotter(shape=(2, 2))
# Show the threshold
p.add_mesh(outline, color="k")
p.add_mesh(threshed, show_scalar_bar=False)
p.camera_position = [-2, 5, 3]
# Show the contour
p.subplot(0, 1)
p.add_mesh(outline, color="k")
p.add_mesh(contours, show_scalar_bar=False)
p.camera_position = [-2, 5, 3]
# Show the slices
p.subplot(1, 0)
p.add_mesh(outline, color="k")
p.add_mesh(slices, show_scalar_bar=False)
p.camera_position = [-2, 5, 3]
# Show the glyphs
p.subplot(1, 1)
p.add_mesh(outline, color="k")
p.add_mesh(glyphs, show_scalar_bar=False)
p.camera_position = [-2, 5, 3]

p.link_views()
p.show()

## Filter Pipeline

In VTK, filters are often used in a pipeline where each algorithm passes
its output to the next filtering algorithm. In PyVista, we can mimic the
filtering pipeline through a chain; attaching each filter to the last
filter. In the following example, several filters are chained together:

1.  First, and empty `threshold` filter to clean out any `NaN` values.
2.  Use an `elevation` filter to generate scalar values corresponding to
    height.
3.  Use the `clip` filter to cut the dataset in half.
4.  Create three slices along each axial plane using the
    `slice_orthogonal` filter.


In [ ]:
# Apply a filtering chain
result = dataset.threshold().elevation().clip(normal="z").slice_orthogonal()

And to view this filtered data, simply call the `plot` method
(`result.plot()`) or create a rendering scene:


In [ ]:
p = pv.Plotter()
p.add_mesh(outline, color="k")
p.add_mesh(result, scalars="Elevation")
p.view_isometric()
p.show()

### Exercise

Let's plot a 2x2 plot of 4 patterns with Sphere set to shrink factors of 0.8, 0.6, 0.4, and 0.2.

In [ ]:
import pyvista as pv

In [ ]:
pv.Sphere?

In [ ]:
pv.DataSet.shrink?

In [ ]:
# Your code here

In [ ]:
mesh1 = pv.Sphere().shrink(shrink_factor=0.8)
mesh2 = pv.Sphere().shrink(shrink_factor=0.6)
mesh3 = pv.Sphere().shrink(shrink_factor=0.4)
mesh4 = pv.Sphere().shrink(shrink_factor=0.2)

p = pv.Plotter(shape=(2, 2))
p.subplot(0, 0)
p.add_text("shrink_factor: 0.8")
p.add_mesh(mesh1)
p.subplot(0, 1)
p.add_text("shrink_factor: 0.6")
p.add_mesh(mesh2)
p.subplot(1, 0)
p.add_text("shrink_factor: 0.4")
p.add_mesh(mesh3)
p.subplot(1, 1)
p.add_text("shrink_factor: 0.2")
p.add_mesh(mesh4)
p.show()